Load our libraries

In [70]:
import json
from pathlib import Path
import mkwikidata # you may need to pip install mkwikidata

Set up our Wikidata query. 

For earlier versions see
https://w.wiki/4wVb
https://w.wiki/4wmP

In [84]:
query = """
SELECT ?typeLabel ?item ?itemLabel  ?inception ?website ?twitter ?GSS ?WDTK WHERE {
  
  {?item wdt:P31/wdt:P279* wd:Q837766 . }
  UNION # Instance of or sub-class of Local Authority or London Borough
  {?item wdt:P31 wd:Q211690 .} 

  ?item wdt:P17 wd:Q145 . #in UK
  ?item wdt:P31 ?type . #get type 
  MINUS {?item wdt:P576 ?abol .} #ignore abolished councils
  MINUS {?item wdt:P31 wd:Q640452 .} #ignore Area Committees 
  MINUS {?item wdt:P31 wd:Q7137435 .} # ignore Parish Councils
  OPTIONAL {?item wdt:P856 ?website .}
  OPTIONAL {?item wdt:P2002 ?twitter .}
  OPTIONAL {?item wdt:P836 ?GSS.}
  OPTIONAL {?item wdt:P8167 ?WDTK .}
  OPTIONAL {?item wdt:P571 ?inception .}
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

Execute the wikidata query, loading the results into a dictionary: query_result. 

In [85]:
query_result = mkwikidata.run_query(query, params={ })

Create a Python Set - wd_set - and load our itemLabels (names of councils)

In [86]:
wd_set = set()

for x in query_result["results"]["bindings"]:
    wd_set.add(x["itemLabel"]["value"])
    #print (x["itemLabel"]["value"])

set the path to where the DC data is. Create a Python set to hold that data

In [87]:
path = Path.cwd().joinpath('data')
dc_council_set = set()


Read the json file into our set - __if__ the council is current (ie not abolished)

In [88]:
with open (path / 'uk_local_authorities.json') as dc_file:
    data = json.load (dc_file)
    for item in data:
        if item['end-date'] =="":
            dc_council_set.add(item['official-name'])

Check how many councils we have in DC set

In [89]:
print(len(dc_council_set))


409


Check how many in our Wikidata set

In [90]:
print(len(wd_set))

499


Using set theory, check how many councils are in DC's set whcih are not in our Wikidata set (derived from the Wikidata Query).

This was 70+ but I've changed the query to add in London Boroughs, and updated all of the NI councils to be Local Authorities. 

I __suspect__ that most of the following are down to Wikidata using _common names_ as the the item title and sometimes giving the _official tital_ as an alias. 

In [96]:
print ("Councils in DC list but not in WD: ", len(dc_council_set - wd_set))
print("======================================")
missing_list = list(dc_council_set - wd_set)
mising_list = missing_list.sort()
for council in missing_list:
    print(council)


Councils in DC list but not in WD:  34
Blackpool Borough Council
Bolton Metropolitan Borough Council
Borough Council of Kings Lynn and West Norfolk
Cambridgeshire and Peterborough Combined Authority
City of Cardiff Council
City of Wolverhampton Council
Folkestone and Hythe District Council
Gateshead Metropolitan Borough Council
Greater London Authority
Greater Manchester Combined Authority
Kirklees Council
Liverpool City Region
London Borough of Hammersmith & Fulham
Mid Ulster District Council
Middlesbrough Borough Council
North Devon District Council
North East Combined Authority
North Northamptonshire Council
North Somerset  Council
North of Tyne Combined Authority
Oldham Metropolitan Borough Council
Royal Borough of Windsor and Maidenhead
Sefton Metropolitan Borough Council
South Yorkshire Mayoral Combined Authority
St Helens Council
Tees Valley Combined Authority
Telford & Wrekin Council
Wakefield Metropolitan District Council
West Midlands Combined Authority
West Northamptonshire 

In [97]:
print ("Councils in WD but not in DC list: ",len(wd_set - dc_council_set))

Councils in WD but not in DC list:  124


The WD query contains parish councils which DC data does not. 

In [95]:
print ("Councils in Wikidata list but not in DC: ", len(wd_set - dc_council_set))
print("======================================")
missing_list = list(wd_set - dc_council_set )
mising_list = missing_list.sort()
for council in missing_list:
    print(council)

Councils in Wikidata list but not in DC:  124
Avon County Council
Aylesbury Vale District Council
Bala Urban District Council
Barking and Dagenham London Borough Council
Barnet London Borough Council
Basildon District Council
Bexley London Borough Council
Blackpool Council
Blaenau Gwent Council
Bolton Council
Borough Council of King's Lynn and West Norfolk
Bournemouth Borough Council
Breckland Council
Brent London Borough Council
Bromley London Borough Council
Camden London Borough Council
Caradon District Council
Cardiff Council
Carrick District Council
Castle Point District Council
Chester-le-Street District Council
Chiltern District Council
Christchurch Borough Council
City of Lincoln District Council
Corby Borough Council
Corby District Council
Croydon London Borough Council
Cumberland County Council
Daventry District Council
Derwentside District Council
Dungannon District Council
Durham City Council
Dyfed County Council
Ealing London Borough Council
Easington District Council
East